In [1]:
import json
import csv
import pandas as pd
from pathlib import Path
from nltk.sentiment.vader import SentimentIntensityAnalyzer


sid = SentimentIntensityAnalyzer()

In [2]:
filePath = Path("yelp_dataset/yelp_academic_dataset_business.json")
df_business = pd.read_json(filePath,lines=True)

df_restaurants = df_business[df_business['categories'].str.contains('Restaurants',na=False)]
restaurant_ids = df_restaurants[['business_id', 'categories']]

In [3]:
review_data = []
counter = 0
filePath = Path('yelp_dataset/yelp_academic_dataset_review.json')
with open(filePath, encoding="utf-8") as file:
    for line in file:
        if counter < 800000:
            review_data.append(json.loads(line))
            counter += 1
        else:
            break

df_review = pd.DataFrame(review_data)

In [4]:
reviews_restaurants = df_review.merge(restaurant_ids, on='business_id')
# reviews_restaurants = reviews_restaurants[['stars','text']]
reviews_restaurants.to_csv('reviews_restaurants.csv')

In [5]:
count = 0
output_rows = []
with open('reviews_restaurants.csv', 'r') as file:
    reader = csv.DictReader(file)

    for row in reader:
        review = row['text']
        stars = row['stars']
        review_id = row['review_id']
        sentiment_score = sid.polarity_scores(review)['compound']
        output_row = {'review_id':review_id,'sentiment': sentiment_score, 'stars': stars}
        output_rows.append(output_row)
# output_df = pd.DataFrame(output_rows)


In [6]:
output_df = pd.DataFrame(output_rows)
# output_df.to_csv("vader_365.csv")
# output_df.to_json("vader_365.json")

In [14]:
output_df = output_df.dropna()
output_df['stars'] = output_df['stars'].str.slice(0,1)
output_df['stars'] = output_df['stars'].astype(int)

In [21]:
a_one_staradj= output_df.loc[(output_df['sentiment'] >= -1) & (output_df['sentiment'] <= -0.5) & (output_df['stars'] == 1), ['review_id','sentiment','stars']]

# Second selection
a_two_staradj = output_df.loc[(output_df['sentiment'] >= -0.5) & (output_df['sentiment'] <= 0.3) & (output_df['stars'] == 2), ['review_id','sentiment','stars']]
a_three_staradj = output_df.loc[(output_df['sentiment'] >= 0.3) & (output_df['sentiment'] <= 0.75) & (output_df['stars'] == 3), ['review_id','sentiment','stars']]
a_four_staradj = output_df.loc[(output_df['sentiment'] >= 0.75) & (output_df['sentiment'] <= 0.9) & (output_df['stars'] == 4), ['review_id','sentiment','stars']]
a_five_staradj = output_df.loc[(output_df['sentiment'] >= 0.9) & (output_df['sentiment'] <= 1) & (output_df['stars'] == 5), ['review_id','sentiment','stars']]


In [10]:
output_df.dtypes

review_id     object
sentiment    float64
stars         object
dtype: object